# Descriptions of Source from Harvard

 
For your reference, here are some descriptions for the fields:
 
For a product for a given year:

- pci: Product Complexity Index: How complex is this product? Is it relatively rare and produced by other diverse/complex countries? 

For a country for a given year:

- eci: Location Complexity Index: How complex is this country’s economy? Intuitively this is an average of the complexities of the products that a location produces.
- coi: Complexity Outlook Index: How well-positioned is this country in the product space? If you are nearby many potential unoccupied complex nodes, you get high COI. Often countries that are located in central / highly connected nodes.

For a specific product in a country for a given year: 

- export_rca: Revealed Comparative Advantage: Does this country export this product more than its “fair share”? i.e. if the word export share of cars is 20% of all exports, and cars compose 40% of your country’s exports, cars have and RCA of 2, i.e. your country exports twice its “fair share” of cars. We say that a country’s produces a product significantly if it has RCA > 1. If a product has RCA <1 for that country, we consider that a potential new product. The feasibility graph you linked us to focuses specifically only on RCA < 1 products.
- distance: How far away is this potential product from my country’s current productive capabilities? 
- cog: Complexity Outlook Gain: How would adding this new product change my complexity outlook? Would it connect me to many new high-complexity nodes?

According to the definitions, following indicators should be in this dataset:

```
- pci        by product, year
- eci        by country, year
- coi        by country, year
- export_rca by product, country, year
- distance   by product, country, year
- cog        by product, country, year
```

Note:

- HS and SITC are different product classifications. So there should 2 product entity

In [1]:
import pandas as pd

In [2]:
import os
import os.path as osp

In [14]:
from ddf_utils.str import format_float_digits
from functools import partial

In [15]:
ff = partial(format_float_digits, digits=6)

In [3]:
source_hs = '../source/gapminder_hs.csv.gz'
source_sitc = '../source/gapminder_sitc.csv.gz'

In [4]:
# data1 = pd.read_csv(source_hs)
data1 = pd.read_csv(source_hs, dtype={'product_code': 'str'})

In [5]:
data1.shape

(6254717, 11)

In [6]:
data1.head()

,year,cog,distance,export_rca,pci,hs_eci,hs_coi,product_name,product_code,location_name,location_code
0,1995,-0.026350,0.986371,0.130978,0.029435,-0.145284,-1.022883,Horses,0101,Aruba,ABW
1,1996,-0.023448,0.992841,0.000000,-0.035006,-0.137835,-1.085390,Horses,0101,Aruba,ABW
2,1997,0.002196,0.993578,0.000000,0.035813,-0.916167,-1.117452,Horses,0101,Aruba,ABW
3,1998,0.039962,0.985613,0.000000,-0.012857,0.458254,-1.003824,Horses,0101,Aruba,ABW
4,1999,0.065873,0.984802,0.061158,0.121131,0.060021,-1.071245,Horses,0101,Aruba,ABW


In [7]:
data1.memory_usage().sum() / 1024 / 1024

524.9168167114258

In [16]:
for i in data1['location_code'].unique():
    if pd.isnull(i):
        print('failed')

In [5]:
# data2 = pd.read_csv(source_sitc)
data2 = pd.read_csv(source_sitc, dtype={'product_code': 'str'})

In [12]:
data2.memory_usage().sum() / 1024 / 1024

745.5780487060547

In [13]:
data2.head()

,year,cog,distance,export_rca,pci,sitc_eci,sitc_coi,product_name,product_code,location_name,location_code
0,1988,-0.094844,0.979881,0.0,-0.226877,1.370849,-0.474975,Animals of the bovine species (including buffa...,0011,Aruba,ABW
1,1989,-0.131799,0.959964,0.0,-0.672922,0.784267,-0.404974,Animals of the bovine species (including buffa...,0011,Aruba,ABW
2,1990,-0.142112,0.980740,0.0,-0.597605,-0.430476,-0.887176,Animals of the bovine species (including buffa...,0011,Aruba,ABW
3,1991,-0.176602,0.987934,0.0,-0.596164,-0.569074,-0.893478,Animals of the bovine species (including buffa...,0011,Aruba,ABW
4,1992,-0.256202,0.978708,0.0,-0.556728,-0.269096,-0.910005,Animals of the bovine species (including buffa...,0011,Aruba,ABW


In [15]:
for i in data2['location_code'].unique():
    if pd.isnull(i):
        print('failed')

In [ ]:
# location entity

In [23]:
cols = ['location_code', 'location_name']
# combine all location in both files
location = pd.concat([data1[cols], data2[cols]], ignore_index=True).drop_duplicates()

In [24]:
location[location.location_code.duplicated()]

,location_code,location_name


In [25]:
location.columns

Index(['location_code', 'location_name'], dtype='object')

In [26]:
location.columns = ['location', 'name']
location['location'] = location['location'].str.lower()

In [27]:
location.head()

,location,name
0,abw,Aruba
27313,afg,Afghanistan
54635,ago,Angola
81957,aia,Anguilla
109270,alb,Albania


In [28]:
location.to_csv('../../ddf--entities--location.csv', index=False)

In [13]:
# HS products

In [17]:
cols = ['product_code', 'product_name']
hs_prod = data1[cols].drop_duplicates()

In [22]:
hs_prod.head()

,product_code,product_name
0,0101,Horses
22,0102,Bovine
44,0103,Swine
66,0104,Sheep
88,0105,Fowl


In [23]:
hs_prod.product_code.hasnans

False

In [24]:
hs_prod.columns = ['product_code_hs', 'name']

In [25]:
hs_prod['product_code_hs'] = hs_prod['product_code_hs'].str.lower()

In [26]:
hs_prod.to_csv('../../ddf--entities--product_code_hs.csv', index=False)

In [39]:
sitc_prod = data2[cols].drop_duplicates()

In [40]:
sitc_prod.head()

,product_code,product_name
0,0011,Animals of the bovine species (including buffa...
29,0012,"Sheep and goats, live"
58,0013,"Swine, live"
87,0014,"Poultry, live"
116,0015,"Equine species, live"


In [41]:
sitc_prod.columns = ['product_code_sitc', 'name']

In [61]:
sitc_prod['product_code_sitc'] = sitc_prod['product_code_sitc'].str.lower()

In [62]:
sitc_prod.to_csv('../../ddf--entities--product_code_sitc.csv', index=False)

In [44]:
# Datapoints

# - pci        by product, year
# - eci        by country, year
# - coi        by country, year
# - export_rca by product, country, year
# - distance   by product, country, year
# - cog        by product, country, year

In [25]:
# pci
cols = ['product_code', 'year', 'pci']
pci_hs = data1[cols].copy()
pci_sitc = data2[cols].copy()

In [7]:
pci_hs.head()

,product_code,year,pci
0,0101,1995,0.029435
1,0101,1996,-0.035006
2,0101,1997,0.035813
3,0101,1998,-0.012857
4,0101,1999,0.121131


In [8]:
pci_hs.sort_values(by=['product_code', 'year']).head()  # all country have same values for given product/year

,product_code,year,pci
0,0101,1995,0.029435
27313,0101,1995,0.029435
54635,0101,1995,0.029435
81957,0101,1995,0.029435
109270,0101,1995,0.029435


In [26]:
pci_hs.columns = ['product_code_hs', 'year', 'pci']
pci_hs = pci_hs.drop_duplicates(subset=['product_code_hs', 'year'])
pci_hs['product_code_hs'] = pci_hs['product_code_hs'].str.lower()
pci_hs['pci'] = pci_hs['pci'].map(ff)
pci_hs.to_csv('../../ddf--datapoints--pci--by--'+'--'.join(list(pci_hs.columns[:-1]))+'.csv', index=False)

In [50]:
# pci_hs.product_code_hs.unique().size

1243

In [52]:
# pci_sitc.product_code.unique().size

787

In [53]:
#!mkdir ../../pci_hs && mkdir ../../pci_sitc

In [10]:
def to_csv(df, g, t, d):
    path = osp.join(d, t.format(g))
    df.to_csv(path, index=False)

In [11]:
# df = pci_hs
# groupby = 'product_code_hs'
# p = '../../pci_hs'

# for g, df_ in df.groupby(groupby):
#     t = 'ddf--datapoints--pci--by--' + groupby + '-{}--year.csv'
#     df__ = df_.dropna(subset=['pci']).copy()
#     df__ = df__.drop_duplicates(subset=[groupby, 'year'])
#     if df__.empty:
#         continue
#     df__[groupby] = df__[groupby].str.lower()
#     to_csv(df__, g, t, p)

In [28]:
pci_sitc.columns = ['product_code_sitc', 'year', 'pci']
pci_sitc = pci_sitc.drop_duplicates(subset=['product_code_sitc', 'year'])
pci_sitc['product_code_sitc'] = pci_sitc['product_code_sitc'].str.lower()
pci_sitc['pci'] = pci_sitc['pci'].map(ff)
pci_sitc.to_csv('../../ddf--datapoints--pci--by--'+'--'.join(list(pci_sitc.columns[:-1]))+'.csv', index=False)

In [67]:
# df = pci_sitc
# groupby = 'product_code_sitc'
# p = '../../pci_sitc'

# for g, df_ in df.groupby(groupby):
#     t = 'ddf--datapoints--pci--by--' + groupby + '-{}--year.csv'
#     df__ = df_.dropna(subset=['pci']).copy()
#     if df__.empty:
#         continue
#     df__[groupby] = df__[groupby].str.lower()
#     to_csv(df__, g, t, p)

In [73]:
# eci

# as we can see below:
# eci/coi are same for a year, no matter which product

In [71]:
data1[(data1.year == 1995) & (data1.location_code == 'ABW')].head()

,year,cog,distance,export_rca,pci,hs_eci,hs_coi,product_name,product_code,location_name,location_code
0,1995,-0.026350,0.986371,0.130978,0.029435,-0.145284,-1.022883,Horses,0101,Aruba,ABW
22,1995,-0.090248,0.987265,0.000000,0.010989,-0.145284,-1.022883,Bovine,0102,Aruba,ABW
44,1995,-0.013347,0.992648,0.000000,0.089716,-0.145284,-1.022883,Swine,0103,Aruba,ABW
66,1995,-0.157109,0.989216,0.000000,-0.559563,-0.145284,-1.022883,Sheep,0104,Aruba,ABW
88,1995,-0.089947,0.991364,0.000000,-0.033183,-0.145284,-1.022883,Fowl,0105,Aruba,ABW


In [72]:
data2[(data2.year == 1995) & (data2.location_code == 'ABW')].head()

,year,cog,distance,export_rca,pci,sitc_eci,sitc_coi,product_name,product_code,location_name,location_code
7,1995,-0.104940,0.979272,0.000000,-0.095885,-0.497029,-1.102927,Animals of the bovine species (including buffa...,0011,Aruba,ABW
36,1995,-0.187256,0.980351,0.000000,-0.594714,-0.497029,-1.102927,"Sheep and goats, live",0012,Aruba,ABW
65,1995,0.066685,0.984902,0.000000,0.128867,-0.497029,-1.102927,"Swine, live",0013,Aruba,ABW
94,1995,-0.048455,0.985503,0.000000,-0.058842,-0.497029,-1.102927,"Poultry, live",0014,Aruba,ABW
123,1995,0.002895,0.980246,0.121011,-0.004677,-0.497029,-1.102927,"Equine species, live",0015,Aruba,ABW


In [74]:
cols = ['location_code', 'year', 'hs_eci']

hs_eci = data1[cols].groupby(['location_code', 'year']).first()

In [76]:
hs_eci = hs_eci.reset_index()
hs_eci.columns = ['location', 'year', 'hs_eci']
hs_eci.location = hs_eci.location.str.lower()

In [77]:
hs_eci.head()

,location,year,hs_eci
0,abw,1995,-0.145284
1,abw,1996,-0.137835
2,abw,1997,-0.916167
3,abw,1998,0.458254
4,abw,1999,0.060021


In [79]:
hs_eci.to_csv('../../ddf--datapoints--hs_eci--by--location--year.csv', index=False)

In [81]:
cols = ['location_code', 'year', 'sitc_eci']

sitc_eci = data2[cols].groupby(['location_code', 'year']).first()

In [91]:
sitc_eci = sitc_eci.reset_index()
sitc_eci.columns = ['location', 'year', 'sitc_eci']
sitc_eci.location = sitc_eci.location.str.lower()

In [92]:
sitc_eci.to_csv('../../ddf--datapoints--sitc_eci--by--location--year.csv', index=False)

In [ ]:
# coi

In [84]:
cols = ['location_code', 'year', 'hs_coi']

hs_coi = data1[cols].groupby(['location_code', 'year']).first()

In [88]:
hs_coi = hs_coi.reset_index()
hs_coi.columns = ['location', 'year', 'hs_coi']
hs_coi.location = hs_coi.location.str.lower()

In [89]:
hs_coi.head()

,location,year,hs_coi
0,abw,1995,-1.022883
1,abw,1996,-1.085390
2,abw,1997,-1.117452
3,abw,1998,-1.003824
4,abw,1999,-1.071245


In [90]:
hs_coi.to_csv('../../ddf--datapoints--hs_coi--by--location--year.csv', index=False)

In [27]:
cols = ['location_code', 'year', 'sitc_coi']

sitc_coi = data2[cols].groupby(['location_code', 'year']).first()

In [28]:
sitc_coi = sitc_coi.reset_index()
sitc_coi.columns = ['location', 'year', 'sitc_coi']
sitc_coi.location = sitc_coi.location.str.lower()

In [29]:
sitc_coi.to_csv('../../ddf--datapoints--sitc_coi--by--location--year.csv', index=False)

In [96]:
# export_rca

In [97]:
cols = ['product_code', 'location_code', 'year', 'export_rca']
hs_erca = data1[cols].copy()

In [99]:
hs_erca.columns = ['product_code_hs', 'location', 'year', 'export_rca']
hs_erca['product_code_hs'] = hs_erca['product_code_hs'].str.lower()
hs_erca['location'] = hs_erca['location'].str.lower()

In [100]:
hs_erca.head()

,product_code_hs,location,year,export_rca
0,0101,abw,1995,0.130978
1,0101,abw,1996,0.000000
2,0101,abw,1997,0.000000
3,0101,abw,1998,0.000000
4,0101,abw,1999,0.061158


In [101]:
!mkdir ../../export_rca_hs && mkdir ../../export_rca_sitc

In [106]:
df = hs_erca
groupby = 'product_code_hs'
p = '../../export_rca_hs'

for g, df_ in df.groupby(groupby):
    t = 'ddf--datapoints--export_rca--by--' + groupby + '-{}--location--year.csv'
    df__ = df_.dropna(subset=['export_rca']).copy()
    if df__.empty:
        continue
    # df__[groupby] = df__[groupby].str.lower()
    df__['export_rca'] = df__['export_rca'].map(ff)
    to_csv(df__, g, t, p)

In [107]:
cols = ['product_code', 'location_code', 'year', 'export_rca']
sitc_erca = data2[cols].copy()

In [108]:
sitc_erca.columns = ['product_code_sitc', 'location', 'year', 'export_rca']
sitc_erca['product_code_sitc'] = sitc_erca['product_code_sitc'].str.lower()
sitc_erca['location'] = sitc_erca['location'].str.lower()

In [109]:
sitc_erca.head()

,product_code_sitc,location,year,export_rca
0,0011,abw,1988,0.0
1,0011,abw,1989,0.0
2,0011,abw,1990,0.0
3,0011,abw,1991,0.0
4,0011,abw,1992,0.0


In [110]:
df = sitc_erca
groupby = 'product_code_sitc'
p = '../../export_rca_sitc'

for g, df_ in df.groupby(groupby):
    t = 'ddf--datapoints--export_rca--by--' + groupby + '-{}--location--year.csv'
    df__ = df_.dropna(subset=['export_rca']).copy()
    if df__.empty:
        continue
    # df__[groupby] = df__[groupby].str.lower()
    df__['export_rca'] = df__['export_rca'].map(ff)
    to_csv(df__, g, t, p)

In [146]:
# clean up memory...

In [137]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# distance

In [150]:
!mkdir ../../distance_hs && mkdir ../../distance_sitc

In [147]:
cols = ['product_code', 'location_code', 'year', 'distance']
hs_distance = data1[cols].copy()

In [148]:
df = hs_distance
col = 'distance'

df.columns = ['product_code_hs', 'location', 'year', col]
df['product_code_hs'] = df['product_code_hs'].str.lower()
df['location'] = df['location'].str.lower()

In [151]:
groupby = 'product_code_hs'
p = '../../distance_hs'

for g, df_ in df.groupby(groupby):
    t = 'ddf--datapoints--' + col + '--by--' + groupby + '-{}--location--year.csv'
    df__ = df_.dropna(subset=[col]).copy()
    if df__.empty:
        continue
    # df__[groupby] = df__[groupby].str.lower()
    df__[col] = df__[col].map(ff)
    to_csv(df__, g, t, p)

In [15]:
cols = ['product_code', 'location_code', 'year', 'distance']
sitc_distance = data2[cols].copy()

In [16]:
df = sitc_distance
col = 'distance'
groupby = 'product_code_sitc'

df.columns = [groupby, 'location', 'year', col]
df[groupby] = df[groupby].str.lower()
df['location'] = df['location'].str.lower()

In [17]:
p = '../../distance_sitc'

for g, df_ in df.groupby(groupby):
    t = 'ddf--datapoints--' + col + '--by--' + groupby + '-{}--location--year.csv'
    df__ = df_.dropna(subset=[col]).copy()
    if df__.empty:
        continue
    # df__[groupby] = df__[groupby].str.lower()
    df__[col] = df__[col].map(ff)
    to_csv(df__, g, t, p)

In [155]:
%reset_selective hs_distance

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [156]:
%reset_selective sitc_distance

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [ ]:
# cog

In [159]:
!mkdir ../../cog_hs && mkdir ../../cog_sitc

In [157]:
cols = ['product_code', 'location_code', 'year', 'cog']
hs_cog = data1[cols].copy()

In [158]:
df = hs_cog
col = 'cog'
groupby = 'product_code_hs'

df.columns = [groupby, 'location', 'year', col]
df[groupby] = df[groupby].str.lower()
df['location'] = df['location'].str.lower()

In [160]:
p = '../../cog_hs'

for g, df_ in df.groupby(groupby):
    t = 'ddf--datapoints--' + col + '--by--' + groupby + '-{}--location--year.csv'
    df__ = df_.dropna(subset=[col]).copy()
    if df__.empty:
        continue
    # df__[groupby] = df__[groupby].str.lower()
    df__[col] = df__[col].map(ff)
    to_csv(df__, g, t, p)

In [8]:
cols = ['product_code', 'location_code', 'year', 'cog']
sitc_cog = data2[cols].copy()

In [11]:
df = sitc_cog
col = 'cog'
groupby = 'product_code_sitc'

df.columns = [groupby, 'location', 'year', col]
df[groupby] = df[groupby].str.lower()
df['location'] = df['location'].str.lower()

In [14]:
p = '../../cog_sitc'

for g, df_ in df.groupby(groupby):
    t = 'ddf--datapoints--' + col + '--by--' + groupby + '-{}--location--year.csv'
    df__ = df_.dropna(subset=[col]).copy()
    if df__.empty:
        continue
    # df__[groupby] = df__[groupby].str.lower()
    df__[col] = df__[col].map(ff)
    to_csv(df__, g, t, p)

In [164]:
# concepts

In [ ]:
import yaml

In [165]:
data1.columns

Index(['year', 'cog', 'distance', 'export_rca', 'pci', 'hs_eci', 'hs_coi',
       'product_name', 'product_code', 'location_name', 'location_code'],
      dtype='object')

In [166]:
data2.columns

Index(['year', 'cog', 'distance', 'export_rca', 'pci', 'sitc_eci', 'sitc_coi',
       'product_name', 'product_code', 'location_name', 'location_code'],
      dtype='object')

In [169]:
measures = ['cog', 'distance', 'export_rca', 'pci', 'hs_eci', 'hs_coi', 'sitc_eci', 'sitc_coi']
entities = ['location', 'product_code_hs', 'product_code_sitc']
time = ['year']
string = ['name', 'domain']

In [172]:
cdf = pd.DataFrame([*measures, *entities, *time, *string], columns=['concept']).set_index('concept')

In [173]:
cdf.loc[measures, 'concept_type'] = 'measure'
cdf.loc[entities, 'concept_type'] = 'entity_domain'
cdf.loc[time, 'concept_type'] = 'time'
cdf.loc[string, 'concept_type'] = 'string'

In [175]:
cdf['domain'] = None

In [176]:
cdf['name'] = None

In [177]:
cdf

,concept_type,domain,name
concept,,,
cog,measure,None,None
distance,measure,None,None
export_rca,measure,None,None
pci,measure,None,None
hs_eci,measure,None,None
hs_coi,measure,None,None
sitc_eci,measure,None,None
sitc_coi,measure,None,None
location,entity_domain,None,None


In [178]:
cdf.to_csv('../../ddf--concepts.csv')

In [179]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
